In [1]:
instance_uri = "https://api.development.sinopia.io/resource/c5a48eb8-a241-4669-8004-40948c1c32dd"
# change the instance_uri to the URI of the instance that you want to serialize for Alma RDF/XML.
# add api key and region for posting the resource to Alma
# https://github.com/LD4P/ils-middleware/wiki/Alma-APIs-in-Airflow

alma_api_key = "your_api_key"
uri_region = "https://api-na.hosted.exlibrisgroup.com"

In [2]:
%pip install rdflib
%pip install lxml
%pip install requests

In [3]:
import requests
from rdflib import Graph, Namespace, URIRef
from rdflib import Namespace, URIRef, RDF, Graph
from rdflib.namespace import RDF
from lxml import etree as ET
from name_space.alma_ns import alma_namespaces


In [4]:
instance_uri = URIRef(instance_uri)
work_uri = None
instance_graph = Graph()
work_graph = Graph()
instance_graph.parse(instance_uri)

<Graph identifier=Nf2f99b2112fe4d53869c4e78ec1b7e93 (<class 'rdflib.graph.Graph'>)>

In [5]:
# Define the bf and bflc namespaces
bf = Namespace("http://id.loc.gov/ontologies/bibframe/")
for prefix, url in alma_namespaces:
    instance_graph.bind(prefix, url)
work_uri = instance_graph.value(subject=URIRef(instance_uri), predicate=bf.instanceOf)
work_uri = URIRef(work_uri)
# Explicitly state that work_uri is of type bf:Work
work_graph.add((work_uri, RDF.type, bf.Work))
# add the work to the instance graph
instance_graph.add((instance_uri, bf.instanceOf, URIRef(work_uri)))
# serialize the instance graph
instance_alma_xml = instance_graph.serialize(format="pretty-xml", encoding="utf-8")

print(instance_graph.serialize(format='pretty-xml'))


<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
  xmlns:bf="http://id.loc.gov/ontologies/bibframe/"
  xmlns:sinopia="http://sinopia.io/vocabulary/"
  xmlns:bflc="http://id.loc.gov/ontologies/bflc/"
  xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
  xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
>
  <bf:Instance rdf:about="https://api.development.sinopia.io/resource/c5a48eb8-a241-4669-8004-40948c1c32dd">
    <rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Print"/>
    <sinopia:hasResourceTemplate>pcc:bf2:Monograph:Instance</sinopia:hasResourceTemplate>
    <bf:instanceOf>
      <rdf:Description rdf:about="https://api.development.sinopia.io/resource/3f2def7e-6f7c-4820-82af-c06a47789f00">
        <rdfs:label xml:lang="en">Snake worship in India</rdfs:label>
      </rdf:Description>
    </bf:instanceOf>
    <bf:title>
      <bf:Title rdf:nodeID="f5b566ccb8632490e9a217e05d362d02cb1">
        <bf:mainTitle xml:lang="en">Snake worship in India</bf:mainTitle>
      <

In [6]:
tree = ET.fromstring(instance_alma_xml)
# apply xslt to normalize instance
xslt = ET.parse("xsl/normalize-instance-sinopia2alma.xsl")
transform = ET.XSLT(xslt)
instance_alma_xml = transform(tree)
instance_alma_xml = ET.tostring(
        instance_alma_xml, pretty_print=True, encoding="utf-8"
        )
# save the xml to a file
with open("alma-instance.xml", "wb") as f:
    f.write(instance_alma_xml)
print(instance_alma_xml.decode("utf-8"))

<bib>
  <record_format>lcbf_instance</record_format>
  <suppress_from_publishing>false</suppress_from_publishing>
  <record>
    <rdf:RDF xmlns:bf="http://id.loc.gov/ontologies/bibframe/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:sinopia="http://sinopia.io/vocabulary/" xmlns:bflc="http://id.loc.gov/ontologies/bflc/" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#">
  <bf:Instance rdf:about="https://api.development.sinopia.io/resource/c5a48eb8-a241-4669-8004-40948c1c32dd" xmlns:bf="http://id.loc.gov/ontologies/bibframe/">
    <rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Print"/>
    <sinopia:hasResourceTemplate>pcc:bf2:Monograph:Instance</sinopia:hasResourceTemplate>
    <bf:instanceOf rdf:resource="https://api.development.sinopia.io/resource/3f2def7e-6f7c-4820-82af-c06a47789f00"/>
    <bf:title>
      <bf:Title rdf:nodeID="f7d6532af99064be584c846ba19a1bce9b1">
        <bf:mainTitle xml:lang="en">Snake worship in India</bf:mainTitle>
      </bf:Tit

In [7]:
# handle 400, an update to the record in Alma
def parse_400(result):
    xml_response = ET.fromstring(result)
    xslt = ET.parse("xsl/put_mms_id.xsl")
    transform = ET.XSLT(xslt)
    result_tree = transform(xml_response)
    put_mms_id_str = str(result_tree)
    print(f"put_mms_id_str: {put_mms_id_str}")
    return put_mms_id_str

# post the instance to Alma
def NewInstancetoAlma():
    with open("alma-instance.xml", "rb") as f:
        data = f.read()

        alma_uri = (
            uri_region
            + "/almaws/v1/bibs?"
            + "from_nz_mms_id=&from_cz_mms_id=&normalization=&validate=false"
            + "&override_warning=true&check_match=false&import_profile=&apikey="
            + alma_api_key
        )
        # post to alma
        alma_result = requests.post(
            alma_uri,
            headers={
                "Content-Type": "application/xml; charset=utf-8",
                "Accept": "application/xml",
                "x-api-key": alma_api_key,
            },
            data=data,
        )
        print(f"alma result: {alma_result.status_code}\n{alma_result.text}")
        result = alma_result.content
        status = alma_result.status_code
        if status == 200:
            xml_response = ET.fromstring(result)
            mms_id = xml_response.xpath("//mms_id/text()")
            print(f"Created record {mms_id}")
        elif status == 400:
            # run xslt on the result in case the response is 400 and we need to update the record
            put_mms_id_str = parse_400(result)
            alma_update_uri = (
                uri_region
                + "/almaws/v1/bibs/"
                + put_mms_id_str
                + "?normalization=&validate=false&override_warning=true"
                + "&override_lock=true&stale_version_check=false&cataloger_level=&check_match=false"
                + "&apikey="
                + alma_api_key
            )
            putInstanceToAlma(
                alma_update_uri,
                data,
            )
        else:
            raise Exception(f"Unexpected status code from Alma API: {status}")

# update the instance in Alma
def putInstanceToAlma(
    alma_update_uri,
    data,
):
    put_update = requests.put(
        alma_update_uri,
        headers={
            "Content-Type": "application/xml; charset=UTF-8",
            "Accept": "application/xml",
        },
        data=data,
    )
    print(f"put update: {put_update.status_code}\n{put_update.text}")
    put_update_status = put_update.status_code
    result = put_update.content
    xml_response = ET.fromstring(result)
    put_mms_id = xml_response.xpath("//mms_id/text()")
    match put_update_status:
        case 200:
            print(f"Updated record {put_mms_id}")
        case 500:
            raise Exception(f"Internal server error from Alma API: {put_update_status}")
        case _:
            raise Exception(
                f"Unexpected status code from Alma API: {put_update_status}"
            )

# Call the function
NewInstancetoAlma()

alma result: 200
<?xml version="1.0" encoding="UTF-8" standalone="yes"?><bib><mms_id>9979213724303681</mms_id><record_format>lc_bf_instance</record_format><linked_record_id/><title>Snake worship in India</title><author>Nair, G. Ravindran</author><network_numbers><network_number>9979213724303681</network_number></network_numbers><date_of_publication>March 1993</date_of_publication><holdings link="https://api-na.hosted.exlibrisgroup.com/almaws/v1/bibs/9979213724303681/holdings"/><created_by>exl_api</created_by><created_date>2024-02-05Z</created_date><last_modified_by>exl_api</last_modified_by><last_modified_date>2024-02-05Z</last_modified_date><suppress_from_publishing>false</suppress_from_publishing><suppress_from_external_search>false</suppress_from_external_search><suppress_from_metadoor>false</suppress_from_metadoor><sync_with_oclc>NONE</sync_with_oclc><sync_with_libraries_australia>NONE</sync_with_libraries_australia><originating_system>01UPENN_INST</originating_system><record><rdf: